In [2]:
# Import the necessary libraries
import numpy as np
import scipy.stats as sts
import requests
import matplotlib.pyplot as plt
import scipy.optimize as opt
import pandas as pd

In [3]:
# Import data
df_2 = pd.read_csv('data/sick.txt').astype('float64')
df_2.head()

,sick,age,children,avgtemp_winter
0,1.67,57.47,3.04,54.10
1,0.71,26.77,1.20,36.54
2,1.39,41.85,2.31,32.38
3,1.37,51.27,2.46,52.94
4,1.45,44.22,2.72,45.90


In [4]:
# Define log likelihood function for normal distribution
def log_lik_norm_2(sick, age, children, temp_winter, beta0, beta1, beta2, beta3, sigma):
    
    epsilon = sick - beta0 - beta1 * age - beta2 * children - beta3 * temp_winter
    pdf_vals = sts.norm.pdf(epsilon, scale=sigma)
    # To avoid "divide by zero encountered in log" warning when optimizing, I replace those 0 pdfs with a small positive number
    pdf_vals[pdf_vals == 0] = 1e-10
    ln_pdf_vals = np.log(pdf_vals)
    log_lik_val = ln_pdf_vals.sum()
    
    return log_lik_val

In [5]:
# Define the criterion function
def crit_2(params, *args):
    
    beta0, beta1, beta2, beta3, sigma = params
    sick, age, children, temp_winter = args
    log_lik_val = log_lik_norm_2(sick, age, children, temp_winter, beta0, beta1, beta2, beta3, abs(sigma))
    neg_log_lik_val = -log_lik_val
    
    return neg_log_lik_val

In [9]:
# minimize the negative sum of the log likelihood
beta1_init_2 = 0.01
beta2_init_2 = 0.4
beta3_init_2 = -0.01
sig_init_2 = 0.5
for beta0_init_2 in np.linspace(0.0001, 1, 10000):
    params_init_2 = np.array([beta0_init_2, beta1_init_2, beta2_init_2, beta3_init_2, sig_init_2])
    MLE_args_2 = (df_2['sick'], df_2['age'], df_2['children'], df_2['avgtemp_winter'])
    results_uncstr_2 = opt.minimize(crit_2, params_init_2, args=(MLE_args_2))
    if round(results_uncstr_2.x[0], 2) == 0.25 and results_uncstr_2.success == True:
        print(results_uncstr_2, beta0_init_2)
        break

C:\Users\Thinkpad\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:1048: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (numpy.dot(yk, sk))
C:\Users\Thinkpad\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:1048: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (numpy.dot(yk, sk))
C:\Users\Thinkpad\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:1048: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (numpy.dot(yk, sk))


      fun: -876.8650462886311
 hess_inv: array([[ 4.31774051e-07, -1.53670021e-08,  1.18942935e-07,
         4.47812306e-10, -7.30556511e-09],
       [-1.53670021e-08,  3.14278602e-09, -2.51600722e-08,
        -1.62109440e-09, -3.91880113e-10],
       [ 1.18942935e-07, -2.51600722e-08,  2.37295095e-07,
         1.18150674e-08,  4.73753887e-09],
       [ 4.47812306e-10, -1.62109440e-09,  1.18150674e-08,
         1.05816982e-09,  3.51207633e-10],
       [-7.30556511e-09, -3.91880113e-10,  4.73753887e-09,
         3.51207633e-10,  2.23558257e-08]])
      jac: array([ 7.62939453e-06, -7.62939453e-06,  7.62939453e-06, -7.62939453e-06,
        7.62939453e-06])
  message: 'Optimization terminated successfully.'
     nfev: 469
      nit: 38
     njev: 67
   status: 0
  success: True
        x: array([ 0.25164638,  0.01293335,  0.40050205, -0.00999167,  0.00301768]) 0.037200000000000004
